# Investigando algo genético para planear torneos *Double Round Robin*

En torneos Double Round Robin todos los equipos se enfrentan 2 veces, una vez en casa y otra de visita.

Como conocemos todas las partidas de antemano, la planeación de un torneo se reduce a ordenar las partidas de una forma válida/óptima. Para que una asignacion sea valida cada equipo debe jugar exactamente una vez por jornada.

Si hay n equipos hay
- n(n-1) juegos en el torneo
- n/2 juegos por jornada
- y 2(n-1) jornadas.

Representamos una partida con la pareja X,Y, que indica que juega X vs. Y en casa de X.

Codigo en ingles. Jornada->week.

In [1]:
import numpy as np
import random,time,json
from tqdm import tqdm
import os

In [2]:
# Given the number of teams, returns the list of games that must be played
def games(nTeams):
    games=[]
    for i in range(1,nTeams+1):
        for j in range(1,nTeams+1):
            if i==j:continue
            games.append((i,j))
    assert len(games)==nTeams*(nTeams-1)
    return games

In [3]:
# Returns a random assignment/tournament by shuffling the list of games
def randomAssignment(nTeams):
    p=games(nTeams)
    random.shuffle(p)
    return p

In [4]:
def lastWeek(assignment,nTeams):
    week=[]
    for i in range(0,len(assignment),nTeams//2):
        week=assignment[i:i+nTeams//2]
    return week

In [5]:
def approxAssignment(nTeams):
    current=[]
    gs=games(nTeams)
    remaining=gs[:]
    while len(current)!=nTeams*(nTeams-1):
        available=[]
        inLastWeek=teamsInGames(lastWeek(current,nTeams))
        for game in remaining:
            if game[0] not in inLastWeek and game[1] not in inLastWeek:
                available.append(game)
        game=random.choice(available) if available else random.choice(remaining)
        current.append(game)
        remaining.remove(game)
    assert len(set(current))==nTeams*(nTeams-1)
    return current

In [6]:
n=4
p=games(n)
p

[(1, 2),
 (1, 3),
 (1, 4),
 (2, 1),
 (2, 3),
 (2, 4),
 (3, 1),
 (3, 2),
 (3, 4),
 (4, 1),
 (4, 2),
 (4, 3)]

In [7]:
randomAssignment(4)

[(4, 1),
 (3, 4),
 (4, 3),
 (2, 4),
 (3, 2),
 (3, 1),
 (2, 1),
 (2, 3),
 (1, 2),
 (1, 3),
 (4, 2),
 (1, 4)]

In [341]:
# Used in conflicts(). Given a list of len 2 tuples, returns a list with all elements.
def teamsInGames(week):
    out=[]
    for game in week:
        out.append(game[0])
        out.append(game[1])
    return out

In [342]:
# Returns the conflicts in an assignment/tournament.
# Marks as 1 games in which a participating team plays more than once in that week.
def conflicts(assignment,nTeams):
    conflicts=[]
    n=nTeams
    for i in range(0,len(assignment),n//2):
        week=assignment[i:i+n//2]
        teamsInWeek=teamsInGames(week)
        for game in week:
            conflicts.append(0)
            for team in game:
                if teamsInWeek.count(team)!=1:
                    conflicts[-1]=1
    return conflicts

In [343]:
p=randomAssignment(4)
p

[(3, 1),
 (4, 1),
 (2, 4),
 (3, 4),
 (1, 4),
 (4, 3),
 (1, 2),
 (1, 3),
 (4, 2),
 (3, 2),
 (2, 3),
 (2, 1)]

In [344]:
conflicts(p,4)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]

In [345]:
nConflicts(p,4)

12

In [346]:
# Returns the number of conflicts in an assignment. 
def nConflicts(assignment,nTeams):
    return sum(conflicts(assignment,nTeams))

## Min conflits (base line)

Para comparar el algo genetico con algo, intente implementar Min Conflicts (ver busqueda local en el lib) con swaps.

In [425]:
# Given the index of a conflict, return the game swap that results least nConflicts.
def minSwap(assignment,conflict,nTeams):
    bestN,bestOthers=None,[]
    for i in range(len(assignment)):
        assignment[conflict],assignment[i]=assignment[i],assignment[conflict]
        n=nConflicts(assignment,nTeams)
        if not bestN or n==bestN:
            bestN=n
            bestOthers.append(i)
        elif bestN and n<bestN:
            bestN=n
            bestOthers=[i]
            
        assignment[conflict],assignment[i]=assignment[i],assignment[conflict]
    return random.choice(bestOthers)

In [426]:
# Returns the index of a random conflict in assignment.
def randomConflict(assignment,nTeams):
    indices=np.argwhere(conflicts(assignment,nTeams))
    return indices[np.random.randint(0,indices.shape[0])][0]

In [446]:
# Min Conflicts implementation.
# Returns final state of current and wether it was solved (nConflicts==0)
def minConflicts(nTeams,maxSteps,iniApprox=True,debug=False,autoEndK=100):
    current=approxAssignment(nTeams) if iniApprox else randomAssignment(nTeams)
    n=nConflicts(current,nTeams)
    lastK=[]
    for i in range(maxSteps):
        n=nConflicts(current,nTeams)
        lastK.append(n)
        lastK=lastK[-autoEndK:]
        if debug:print(n)
        if n==0 or (lastK and lastK.count(n)==autoEndK):break
        conflict=randomConflict(current,nTeams)
        other=minSwap(current,conflict,nTeams)
        current[conflict],current[other]=current[other],current[conflict]
    return current,i#i!=maxSteps-1

In [473]:
def kMinConflictsSolutions(n,k):
    solutions=[]
    avrgTime,avrgSteps,avrgResets=0,0,0
    maxSteps=n*15
    while len(solutions)<k:
        start=time.time()
        sol,steps=minConflicts(n,maxSteps,debug=False)
        t=time.time()-start
        if nConflicts(sol,n)==0:
            solutions.append(tuple(sol))
            avrgTime+=t
            avrgSteps+=steps
        else:
            avrgResets+=1
    return solutions,list(set(solutions)),avrgTime/k,avrgSteps/k,avrgResets/k,maxSteps

In [482]:
for n in range(20,32,2):
    sols,unique,t,steps,resets,maxSteps=kMinConflictsSolutions(n,100)
    with open("minConflictsSols/"+str(n)+".json","w") as f:
        f.write(json.dumps(sols))
    print(n,len(unique),t,steps,resets,maxSteps)

KeyboardInterrupt: 

In [483]:
minConflicts(32,1000,debug=True)

88
87
85
83
82
78
76
75
73
71
70
68
66
64
63
63
63
62
60
60
60
59
57
56
55
55
54
54
53
53
53
53
52
51
51
51
49
49
49
47
47
46
45
45
45
45
45
43
43
43
43
43
40
40
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
38
37
36
36
36
34
34
34
34
33
33
33
33
30
30
30
30
28
28
28
28
28
28
28
28
28
28
28
28
28
28
26
26
26
26
24
24
24
24
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
22
21
20
20
18
18
18
18
16
16
16
14
14
14
14
14
14
14
14
14
14
13
13
13
13
13
13
12
12
12
12
12
12
10
10
10
10
10
10
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
6
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4
4


([(30, 11),
  (32, 9),
  (3, 7),
  (13, 2),
  (20, 29),
  (8, 22),
  (19, 5),
  (10, 24),
  (27, 14),
  (4, 1),
  (21, 18),
  (25, 17),
  (23, 15),
  (6, 28),
  (16, 12),
  (31, 26),
  (30, 18),
  (24, 20),
  (7, 17),
  (3, 5),
  (27, 19),
  (11, 22),
  (8, 2),
  (32, 28),
  (23, 14),
  (9, 10),
  (29, 31),
  (26, 15),
  (4, 6),
  (16, 25),
  (12, 13),
  (1, 21),
  (20, 8),
  (11, 9),
  (17, 12),
  (13, 3),
  (24, 18),
  (31, 25),
  (19, 16),
  (5, 1),
  (32, 30),
  (4, 23),
  (21, 7),
  (2, 15),
  (26, 27),
  (28, 10),
  (22, 6),
  (14, 29),
  (31, 3),
  (10, 30),
  (22, 24),
  (14, 16),
  (29, 25),
  (26, 17),
  (11, 5),
  (6, 27),
  (9, 8),
  (7, 2),
  (13, 32),
  (21, 19),
  (18, 23),
  (12, 15),
  (28, 4),
  (20, 1),
  (30, 31),
  (2, 19),
  (4, 10),
  (25, 3),
  (16, 8),
  (15, 22),
  (11, 14),
  (18, 5),
  (28, 13),
  (20, 7),
  (23, 12),
  (1, 27),
  (6, 21),
  (29, 26),
  (24, 9),
  (32, 17),
  (17, 7),
  (14, 21),
  (13, 9),
  (3, 20),
  (32, 15),
  (29, 2),
  (12, 1),
  (16,

In [418]:
# Average nConflicts per nTeams of randomAssignment and 
n=6
s=0
for n in range(4,32,2):
    a=0
    s=0
    for i in range(100):
        s+=nConflicts(randomAssignment(n),n)
        a+=nConflicts(approxAssignment(n),n)
        
    print(n,s/100,a/100,n*(n-1),(s/100)/(n*(n-1)),(a/100)/(n*(n-1)))

4 9.76 0.0 12 0.8133333333333334 0.0
6 24.59 5.29 30 0.8196666666666667 0.17633333333333334
8 47.65 9.65 56 0.8508928571428571 0.17232142857142857
10 76.86 15.68 90 0.854 0.17422222222222222
12 112.26 22.46 132 0.8504545454545455 0.17015151515151516
14 155.02 31.0 182 0.8517582417582418 0.17032967032967034
16 205.75 36.44 240 0.8572916666666667 0.15183333333333332
18 263.11 42.8 306 0.8598366013071896 0.13986928104575164
20 327.54 51.93 380 0.8619473684210527 0.13665789473684212
22 397.01 58.68 462 0.8593290043290043 0.12701298701298702
24 473.98 68.1 552 0.8586594202898551 0.1233695652173913
26 557.79 76.77 650 0.8581384615384615 0.1181076923076923
28 650.39 85.33 756 0.8603042328042327 0.11287037037037037
30 746.78 94.95 870 0.858367816091954 0.10913793103448276


## Genetic algo

Tenemos que definir operadores de mutacion y de reproduccion. Lo que se me ocurrio es que el de mutacion puede ser un swap aleatorio y el de cruzamiento.

In [513]:
def mutate(assignment):
    # Random swap
    i,j=np.random.choice(len(assignment),size=2,replace=False)
    assignment=assignment[:]
    assignment[i],assignment[j]=assignment[j],assignment[i]
    return assignment

In [520]:
def freeMutate(assignment,nTeams):
    a=assignment[:]
    k=random.randint(0,len(a)-1)
    a[k]=random.choice(games(nTeams))
    return a

In [521]:
def crossover(a,b):
    assert len(a)==len(b),"a and b must be same length"
    k=random.randint(0,len(a)) # Random crossover point
    #print(k)
    offspring1=a[:k]
    for game in b:
        if len(offspring1)==len(a):break
        if game not in offspring1:
            offspring1.append(game)
    
    #print(offspring1,len(offspring1))
    assert len(offspring1)==len(a)
    return offspring1

In [522]:
def freeCrossover(a,b):
    assert len(a)==len(b),"a and b must be same length"
    k=random.randint(0,len(a)) # Random crossover point
    offspring1=a[:k]+b[k:]
    offspring2=b[:k]+a[k:]
    assert len(offspring1)==len(a) and len(offspring2)==len(a)
    return offspring1,offspring2

In [557]:
def hamming(a,b):
    dist=0
    assert len(a)==len(b)
    for i in range(len(a)):
        if a[i]!=b[i]:
            dist+=1
    return dist

In [558]:
def hammingOfList(l):
    dist=0
    for i in range(len(l)):
        for j in range(i+1,len(l)):
            dist+=hamming(l[i],l[j])
    return dist

In [523]:
a=randomAssignment(5)
b=randomAssignment(5)

In [8]:
# Given an assignment, returns the sum of the time between games of pairs of teams
def timeBetweenPairs(assignment,nTeams):
    time=0
    # For every pair of teams
    for i in range(1,nTeams+1):
        for j in range(i+1,nTeams+1):
            if i==j:continue
            #print(i,j)
            gameA=assignment.index((i,j))
            gameB=assignment.index((j,i))
            time+=max(gameA,gameB)-min(gameA,gameB)
    return time

In [9]:
a=randomAssignment(4)
a

[(1, 2),
 (4, 2),
 (3, 2),
 (3, 4),
 (3, 1),
 (2, 1),
 (2, 3),
 (1, 3),
 (1, 4),
 (4, 3),
 (4, 1),
 (2, 4)]

In [14]:
len(a)==4*(4-1)

True

In [11]:
timeBetweenPairs(a,4)

30

In [12]:
import itertools

In [22]:
maxTeorico=[(1,2),(1,3),(1,4),(2,3),(2,4),(3,4),(4,3),(4,2),(3,2),(4,1),(3,1),(2,1)]
j=len(maxTeorico)
(j/2)**2,timeBetweenPairs(maxTeorico,4)

(36.0, 36)

In [29]:
m=0
mejores=None
for _ in range(1000000):
    a=randomAssignment(nTeams=4)
    t=timeBetweenPairs(i,4)
    if t>m:
        m=t
        mejores=[i]
    elif t==m:
        mejores.append(i)

In [30]:
m,mejores

(14,
 [((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2),
   (4, 3),
   (1, 3),
   (2, 3),
   (3, 1),
   (4, 2),
   (2, 4)),
  ((1, 2),
   (1, 4),
   (4, 1),
   (3, 4),
   (2, 1),
   (3, 2)

In [524]:
# Fitness function with which to eval individuals and determine their survaival.
# At first, we will only consider the number of conflicts.
# We can then add other soft requirements
def fitness(individual,nTeams):
    return -nConflicts(individual,nTeams)

In [559]:
# Genetic algo
nTeams=16
n=500 #size of population
nElite=int(n*0.05) #proportion best of population to retain
k=0.80 # can regulate temperature
mutationRate=0.2 # rate at which to mutate individuals
assert n%2==0,'N must be even to crossover'
# Random inicialitation of population
population=[approxAssignment(nTeams) for i in range(n)]
scores=[fitness(ind,nTeams) for ind in population]
for gen in range(5000): # Change with other stopping critiria (convergence, etc.)
    
    #min(0.5+(gen/1000),0.95)
    # SELECTION (Investigate other methods)
    # Elitist -> choose best n
    elite=sorted(population,key=lambda x:fitness(x,nTeams),reverse=True)[:nElite]
    
    # Tournament with remaining n-nElite
    fromTournament=[]
    for _ in range(n-nElite):
        a,b=np.random.choice(n,size=2,replace=False)
        best,worst=(a,b) if fitness(population[a],nTeams)>fitness(population[b],nTeams) else (b,a)
        if random.random()<k:
            fromTournament.append(population[best])
        else:
            fromTournament.append(population[worst])
    
    # CROSSOVER -> according to fitness
    #for i in range(0,len(population),2):
        #population.append(crossover(population[i],population[i+1]))
    #for i in range(0,len(population),2):
        #population.append(crossover(population[i],population[i+1]))
    #for i in range(len(population)//2):
        #population.append(crossover(population[i],population[len(population)-1-i]))
    
    population=[]
    for _ in range(n-nElite):
        a,b=np.random.choice(len(fromTournament),size=2,replace=False)
        population.append(freeCrossover(fromTournament[a],fromTournament[b])[0])
    
    """
    population=[] 
    scores=[fitness(ind,nTeams) for ind in fromTournament]
    temp=[1/-i if i<0 else 2 for i in scores]
    s=sum(temp)
    ps=[i/s for i in temp]
    #print(scores)
    for i in range(len(fromTournament)):
        a,b=np.random.choice(len(fromTournament),size=2,replace=False,p=ps)
        population.append(crossover(fromTournament[a],fromTournament[b])) 
    """
    """
    scores=[fitness(ind,nTeams) for ind in population]
    temp=[1/-i if i<0 else 2 for i in scores]
    s=sum(temp)
    ps=[i/s for i in temp]
    #print(scores)
    for i in range(int(len(population)**0.5)):
        a,b=np.random.choice(n,size=2,replace=False,p=ps)
        population.append(crossover(population[a],population[b]))
    """
     
    # MUTATION
    for i in range(len(population)):
        if random.random()<mutationRate:
            population[i]=freeMutate(population[i],nTeams)
    
    #Add elite
    population=elite+population
    assert len(population)==n
    
    # LOGGING
    scores=[fitness(ind,nTeams) for ind in population]
    #print(','.join([str(int(i)) for i in scores]),np.array(scores).mean())
    print(gen,k,hammingOfList(population),np.array(scores).mean(),np.array(scores).min(),np.array(scores).max(),(np.array(scores)==0).sum())
    
    # STOPPING CRITIREA -> if 90% of population are valid sols. (Change later)
    if (np.array(scores)==0).sum()>=len(population)*0.9:
        break

print(gen,"generations")

0 0.8 29694444 -36.954 -61 -12 0
1 0.8 29574466 -37.752 -60 -12 0
2 0.8 29398719 -38.338 -61 -12 0
3 0.8 29165461 -38.296 -65 -12 0
4 0.8 28923241 -37.292 -66 -12 0
5 0.8 28614966 -37.194 -64 -12 0
6 0.8 28218559 -37.342 -67 -12 0
7 0.8 27722113 -35.178 -62 -11 0
8 0.8 27071370 -35.2 -66 -11 0
9 0.8 26197669 -34.274 -70 -11 0
10 0.8 24957989 -32.9 -61 -11 0
11 0.8 24054779 -32.448 -67 -11 0
12 0.8 22714230 -29.668 -63 -11 0
13 0.8 21528381 -27.734 -57 -9 0
14 0.8 21052811 -27.966 -60 -9 0
15 0.8 20541442 -27.152 -53 -9 0
16 0.8 20349041 -26.882 -57 -9 0
17 0.8 19771084 -26.316 -56 -9 0
18 0.8 18924192 -25.434 -55 -9 0
19 0.8 17907860 -24.058 -52 -9 0
20 0.8 16561591 -23.434 -55 -9 0
21 0.8 14337746 -22.022 -56 -9 0
22 0.8 12735618 -20.282 -54 -9 0
23 0.8 9359655 -18.196 -52 -9 0
24 0.8 8001140 -16.476 -59 -7 0
25 0.8 6081968 -15.258 -42 -7 0
26 0.8 4693792 -14.12 -38 -7 0
27 0.8 3941762 -13.096 -34 -7 0
28 0.8 3832063 -13.022 -33 -7 0
29 0.8 3595784 -12.352 -28 -7 0
30 0.8 3373600 -11.

268 0.8 117228 -5.204 -13 -4 0
269 0.8 106966 -5.112 -13 -4 0
270 0.8 87632 -4.922 -11 -4 0
271 0.8 102953 -5.044 -12 -4 0
272 0.8 103439 -5.048 -12 -4 0
273 0.8 116410 -5.182 -11 -4 0
274 0.8 111295 -5.154 -12 -4 0
275 0.8 118746 -5.21 -11 -4 0
276 0.8 108901 -5.116 -13 -4 0
277 0.8 104534 -5.068 -13 -4 0
278 0.8 99017 -4.982 -11 -4 0
279 0.8 90085 -4.872 -10 -4 0
280 0.8 90630 -4.942 -12 -4 0
281 0.8 96067 -4.996 -15 -4 0
282 0.8 95500 -5.0 -13 -4 0
283 0.8 112939 -5.172 -12 -4 0
284 0.8 113322 -5.174 -15 -4 0
285 0.8 122260 -5.282 -15 -4 0
286 0.8 113817 -5.188 -15 -2 0
287 0.8 102998 -5.03 -13 -2 0
288 0.8 115771 -5.1 -16 -2 0
289 0.8 124169 -5.054 -13 -2 0
290 0.8 138224 -4.794 -13 -2 0
291 0.8 150201 -4.728 -13 -2 0
292 0.8 158369 -4.36 -12 -2 0
293 0.8 160834 -4.24 -14 -2 0
294 0.8 159748 -3.918 -14 -2 0
295 0.8 146839 -3.578 -15 -2 0
296 0.8 135111 -3.34 -13 -2 0
297 0.8 129791 -3.294 -11 -2 0
298 0.8 125858 -3.286 -11 -2 0
299 0.8 113908 -3.194 -12 -2 0
300 0.8 109563 -3.164 -

KeyboardInterrupt: 

In [560]:
unique=set()
for i in population:
    if nConflicts(i,nTeams)==0:
        unique.add(tuple(i))

In [561]:
len(unique)

2

In [549]:
for i in unique:
    print(i)
    for team in teamsInGames(list(i)):
        if teamsInGames(list(i)).count(team)!=nTeams:
            print(team,teamsInGames(list(i)).count(team),len(i))
    print()

((11, 5), (2, 14), (15, 7), (4, 3), (8, 13), (12, 10), (1, 16), (6, 9), (5, 4), (7, 14), (3, 12), (1, 6), (16, 9), (13, 11), (2, 8), (10, 15), (2, 5), (3, 8), (7, 15), (16, 13), (11, 4), (10, 14), (12, 9), (6, 1), (11, 16), (5, 3), (9, 13), (6, 4), (8, 12), (10, 7), (15, 14), (2, 1), (9, 5), (16, 8), (7, 10), (1, 15), (6, 2), (13, 3), (11, 12), (14, 4), (2, 12), (5, 7), (4, 10), (9, 14), (3, 1), (16, 6), (15, 8), (11, 13), (12, 1), (6, 8), (2, 16), (4, 14), (10, 3), (15, 13), (7, 5), (11, 9), (7, 3), (8, 10), (15, 9), (5, 12), (13, 6), (16, 2), (11, 14), (4, 1), (8, 11), (4, 7), (3, 15), (10, 12), (13, 9), (14, 1), (5, 16), (2, 6), (3, 9), (4, 8), (13, 14), (2, 7), (1, 12), (15, 10), (5, 11), (6, 16), (16, 5), (10, 4), (12, 6), (8, 2), (9, 1), (15, 11), (7, 13), (3, 14), (7, 11), (3, 10), (5, 8), (6, 14), (12, 16), (9, 2), (13, 4), (15, 1), (12, 5), (16, 10), (14, 13), (4, 15), (11, 3), (8, 1), (6, 7), (2, 9), (6, 10), (14, 16), (12, 4), (11, 7), (1, 2), (8, 9), (15, 3), (5, 13), (1, 1